# Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  # optimizer used to update the weights

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

# Checking the GPU availability and properties

In [2]:
# Check for detecting if GPU is available
print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.is_available())

# GPU device details
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

torch.set_default_tensor_type('torch.cuda.FloatTensor')

# setting the device variable to use GPU if available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

1.7.0
0.8.1
True
0
1
GeForce GTX 1650


# E-T-L Step

Extract ==> Transform ==> Load

In [3]:

# Extracting/Downloading and transforming the images to tensor
train_set = torchvision.datasets.EMNIST(
    root='./data/EMNIST'
    ,split='byclass'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

# Loading the training images in the form of batches.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True)

# Examine the training data

In [4]:
len(train_set)

697932

In [5]:
print('Max Label id: ',train_set.train_labels.max().item())
print('Min Label id: ',train_set.train_labels.min().item())
print('Images in each label: ',train_set.train_labels.bincount())



Max Label id:  61
Min Label id:  0
Images in each label:  tensor([34585, 38374, 34203, 35143, 33535, 31416, 34232, 35754, 33946, 33847,  6407,  3878, 10094,  4562,  4934,  9182,
         2517,  3152, 11946,  3762,  2468,  5076,  9002,  8237, 24983,  8347,  2605,  5073, 20764,  9820, 12602,  4637,
         4695,  2771,  4743,  2701, 10033,  5159,  2854, 10177, 24631,  2561,  3687,  8738,  2725,  1896,  2491, 15318,
         2645, 11418,  2749,  2448,  2994, 14105,  2699, 18262,  2830,  2910,  2697,  2822,  2365,  2725],
       device='cpu')


In [6]:
sample = next(iter(train_set))
image, label = sample 

image.shape, label

(torch.Size([1, 28, 28]), 35)

# Design the training model

In [7]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=30, out_channels=62, kernel_size=3, padding=1)


    def forward(self, t):                                           #input size >> output size >> receptive field
        # (1) Input layer
        t=t                                                         #1x28x28 >> 1x28x28 >> 0

        # (2) hidden conv layer
        t=self.conv1(t)                                             #1x28x28 >> 10x28x28 >> 3
        t=F.relu(t)    

        # (3) hidden conv layer
        t=self.conv2(t)                                             #10x28x28 >> 10x28x28 >> 5 
        t=F.relu(t)
        t=F.max_pool2d(t, kernel_size=2, stride=2)                  #10x28x28 >> 10x14x14 >> 10

        # (4) hidden conv layer
        t=self.conv3(t)                                             #10x14x14 >> 20x14x14 >> 12
        t=F.relu(t)    

        # (5) hidden conv layer
        t=self.conv4(t)                                             #20x14x14 >> 20x14x14 >> 14
        t=F.relu(t)
        t=F.max_pool2d(t, kernel_size=2, stride=2)                  #20x14x14 >> 20x7x7 >> 28

        # (6) hidden conv layer
        t=self.conv5(t)                                             #20x7x7 >> 30x7x7 >> 30
        t=F.relu(t)    

        # (7) hidden conv layer
        t=self.conv6(t)                                             #30x7x7 >> 62x7x7 >> 32

        # (8) output layer
        t=F.adaptive_avg_pool2d(t, (1, 1))                          #62x7x7 >> 62x1x1 
        t=t.squeeze()                                               #62x1x1 >> 62
        return t





# Verifying the Network output

In [8]:
network1 = Network()
network1.to(device)

image = image.reshape(1,1,28,28)
image = image.to(device)
t1 = network1(image)
print(t1.shape)
t1

torch.Size([62])


tensor([ 0.0006, -0.0251,  0.0529,  0.0658,  0.0757, -0.0080,  0.0377, -0.0408, -0.0607, -0.0263, -0.0050, -0.0620,
        -0.0209, -0.0246, -0.0523,  0.0619,  0.0031,  0.0220,  0.0417, -0.0140,  0.0629,  0.0368,  0.0492,  0.0819,
         0.0468, -0.0538,  0.0130, -0.0341,  0.0007,  0.0638,  0.0143,  0.0128,  0.0400,  0.0377, -0.0140, -0.0446,
         0.0558,  0.0193,  0.0055,  0.0321,  0.0551, -0.0318,  0.0426, -0.0188,  0.0263, -0.0137, -0.0343, -0.0323,
        -0.0234, -0.0203,  0.0121,  0.0659,  0.0079, -0.0441,  0.0222, -0.0132, -0.0139,  0.0339,  0.0311,  0.0387,
        -0.0300,  0.0147], grad_fn=<SqueezeBackward0>)

In [9]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

# Training the neural network 

To train a network, need to follow the following process

1. create a batch of images
2. calculate the loss function(cross entropy loss function)
3. calculate the gradient(backprop)
4. update the weights to reduce the loss(optimizer)
5. repeat step 1 to 4 for all the batches untill you reach 1 epoch

In [10]:
network = Network()
network.to(device)

optimizer=optim.Adam(network.parameters(), lr=0.01)  # Using adam optimizer

for epoch in range(20):      # 1 epoch is training the network through all the images once, we have 20 epochs here 
    
    total_loss =0
    total_correct=0

    for batch in train_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        preds = network(images) # pass batch of images through the network
        loss=F.cross_entropy(preds, labels) # calculate the loss

        # pytorch accumulates the gradients
        # so zero out the gradients before calculating again
        optimizer.zero_grad() 
        loss.backward() # calculate gradient
        optimizer.step() # update weights

        total_loss += loss
        total_correct += get_num_correct(preds,labels)

    print("epoch:", epoch, "total correct:", total_correct, 'total loss:', total_loss)




epoch: 0 total correct: 503118 total loss: tensor(2498.7554, grad_fn=<AddBackward0>)
epoch: 1 total correct: 570040 total loss: tensor(1470.7184, grad_fn=<AddBackward0>)
epoch: 2 total correct: 574923 total loss: tensor(1402.8359, grad_fn=<AddBackward0>)
epoch: 3 total correct: 577396 total loss: tensor(1370.0636, grad_fn=<AddBackward0>)
epoch: 4 total correct: 578769 total loss: tensor(1347.6989, grad_fn=<AddBackward0>)
epoch: 5 total correct: 579763 total loss: tensor(1333.6880, grad_fn=<AddBackward0>)
epoch: 6 total correct: 580950 total loss: tensor(1321.4247, grad_fn=<AddBackward0>)
epoch: 7 total correct: 581647 total loss: tensor(1309.9242, grad_fn=<AddBackward0>)
epoch: 8 total correct: 582040 total loss: tensor(1302.4614, grad_fn=<AddBackward0>)
epoch: 9 total correct: 582975 total loss: tensor(1293.4292, grad_fn=<AddBackward0>)
epoch: 10 total correct: 583089 total loss: tensor(1287.7362, grad_fn=<AddBackward0>)
epoch: 11 total correct: 583535 total loss: tensor(1278.7551, gr

In [11]:
#Accuracy
total_correct/len(train_set)

0.8400933042187491